## 簡介 ##
此代碼用來讓LLM根據表格資料與使用者的提問要求，透過pipline與tree stucture，生成報導或分析資料

此篇研究只需提供
"main.txt"為使用者的大綱與簡短想法
"data_description.txt"為要分析的table columns所代表的意義
就可產生完整報導
(可使用在產生任何報導上不限於羽球)

# STEP 1

刪減不必要的columns

結果保留['rally', 'time', 'roundscore_A', 'roundscore_B', 'player', 'type', 'lose_reason', 'getpoint_player']

In [28]:
import os
api_key = os.getenv("Gemini_API")
if not api_key:
    print("❌ Gemini_API 環境變數未設定")

In [30]:
#正式
import dspy
import json
import re
from typing import List, Dict, Any, Optional, ClassVar
import os
from dataclasses import dataclass
import pandas as pd
from openai import OpenAI

class GeminiOpenAI(dspy.LM):
    def __init__(self, api_key, model_name="gemini-2.0-flash"):
        self.api_key = api_key
        self.model_name = model_name
        # 使用 Google 的 OpenAI 兼容端點
        self.client = OpenAI(
            api_key=api_key,
            base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
        )
        super().__init__(model=model_name)
     
    def __call__(self, messages=None, **kwargs):
        if messages is None:
            raise ValueError("Missing 'messages' argument")
         
        # Convert messages to OpenAI format
        if isinstance(messages, list):
            formatted_messages = []
            for msg in messages:
                if isinstance(msg, dict) and 'content' in msg:
                    role = msg.get('role', 'user')
                    formatted_messages.append({
                        'role': role,
                        'content': msg['content']
                    })
                else:
                    formatted_messages.append({
                        'role': 'user',
                        'content': str(msg)
                    })
        else:
            formatted_messages = [{'role': 'user', 'content': str(messages)}]
         
        try:
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=formatted_messages,
                **kwargs
            )
            
            if not response.choices or not response.choices[0].message.content:
                raise ValueError("Empty response from Gemini")
            
            return [{
                'text': response.choices[0].message.content,
                'logprobs': None
            }]
        except Exception as e:
            print(f"Error from Gemini model: {e}")
            return [{
                'text': "⚠️ Gemini API 回應失敗,可能已達限額或出現錯誤。",
                'logprobs': None
            }]
     
    def basic_request(self, prompt, **kwargs):
        try:
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=[{'role': 'user', 'content': prompt}],
                **kwargs
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error from Gemini model: {e}")
            return "⚠️ 無法取得 Gemini 回應"

def setup_gemini_api(api_key, model_name="gemini-2.0-flash"):
    lm = GeminiOpenAI(api_key=api_key, model_name=model_name)
    dspy.settings.configure(lm=lm)
    return lm

def read_text_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except UnicodeDecodeError:
        with open(file_path, 'r', encoding='latin1') as file:
            return file.read()

def parse_list_from_response(response_text: str) -> List[str]:
    """
    Parse a Python list from various response formats including markdown code blocks
    """
    if not response_text or response_text.strip() == "":
        print("⚠️ 回應為空")
        return []
    
    # Remove leading/trailing whitespace
    text = response_text.strip()
    
    # Remove markdown code blocks
    text = re.sub(r'```(?:python|json)?\s*', '', text)
    text = re.sub(r'```\s*', '', text)
    
    # Remove any additional backticks
    text = text.strip('`').strip()
    
    # Try to find a list pattern in the text
    list_match = re.search(r'\[.*?\]', text, re.DOTALL)
    
    if list_match:
        list_text = list_match.group(0)
    else:
        print(f"⚠️ 無法在回應中找到列表格式")
        print(f"完整回應: {text[:200]}...")
        return []
    
    # Clean up the list text
    list_text = list_text.strip()
    
    # Try multiple parsing strategies
    try:
        # Strategy 1: Parse as-is
        return json.loads(list_text)
    except json.JSONDecodeError:
        pass
    
    try:
        # Strategy 2: Convert single quotes to double quotes
        list_text_double = list_text.replace("'", '"')
        return json.loads(list_text_double)
    except json.JSONDecodeError:
        pass
    
    try:
        # Strategy 3: Manual parsing for simple cases
        # Remove brackets and split by comma
        content = list_text.strip('[]').strip()
        if not content:
            return []
        
        # Split by comma and clean each item
        items = []
        for item in content.split(','):
            item = item.strip().strip('"').strip("'").strip()
            if item:
                items.append(item)
        
        if items:
            print(f"✓ 使用手動解析成功")
            return items
    except Exception as e:
        print(f"⚠️ 手動解析失敗: {e}")
    
    print(f"❌ 所有解析方法都失敗了")
    print(f"原始文本: {list_text[:200]}")
    return []


def extract_news_relevant_fields(description_path: str, main_path: str, model_name="gemini-2.0-flash"):
    """
    從描述文件和大綱文件中提取相關欄位
    
    Args:
        description_path: 資料欄位描述文件路徑
        main_path: 大綱文件路徑
        model_name: 使用的模型名稱
    
    Returns:
        List[str]: 篩選出的欄位列表
    """
     
    lm = setup_gemini_api(api_key, model_name)
    main_content = read_text_file(main_path)
    description = read_text_file(description_path)
    
    prompt = f"""Using the following outline and list of data column descriptions, select only the columns that are useful for the outline.

## outline
{main_content}

## Data Column Descriptions:
{description}

---

Please return only a Python list of column names, like this:
['player_name', 'match_score', 'duration', ...]

Do not include explanations or any other text. Return only the list."""
     
    result = lm.basic_request(prompt)
    
    print(f"🔍 原始回應:\n{result}\n")
    
    selected_fields = parse_list_from_response(result)
    
    if selected_fields:
        print("✅ 篩選出的欄位:", selected_fields)
    else:
        print("❌ 未能成功解析欄位列表")
    
    return selected_fields

In [17]:
# 直接調用函式
fields = extract_news_relevant_fields("data_description.txt", "main.txt")
print("最終欄位清單:", fields)


🔍 原始回應:
```python
['rally', 'time', 'roundscore_A', 'roundscore_B', 'player', 'type', 'lose_reason', 'getpoint_player']
```

✅ 篩選出的欄位: ['rally', 'time', 'roundscore_A', 'roundscore_B', 'player', 'type', 'lose_reason', 'getpoint_player']
最終欄位清單: ['rally', 'time', 'roundscore_A', 'roundscore_B', 'player', 'type', 'lose_reason', 'getpoint_player']


In [18]:
df = pd.read_csv("set1.csv")
filtered_df = df[fields]
filtered_df.to_csv("filtered_set1.csv")

In [19]:
def extract_descriptions_for_fields(fields: List[str], desc_path: str, output_path: str):
    description_text = read_text_file(desc_path)

    field_desc = {}
    for line in description_text.splitlines():
        for field in fields:
            if line.lower().startswith(field.lower() + ":"):
                field_desc[field] = line.strip()

    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            for field in fields:
                f.write(field_desc.get(field, f"{field}: [Description not found]") + "\n")
        print(f"✅ 已將欄位描述寫入 {output_path}")
    except Exception as e:
        print(f"❌ 寫入失敗: {e}")


extract_descriptions_for_fields(fields, 'data_description.txt', "filtered_data_description.txt")

✅ 已將欄位描述寫入 filtered_data_description.txt


# STEP 2

藉由人為輸入問題與方向提示，給LLM做完整分析問題與方向

In [22]:
def generate_chain_of_thought_response(main_path: str, desc_path: str, output_path: str, model_name="gemini-2.0-flash"):
    """
    生成 Chain-of-Thought 分析回應
    
    Args:
        main_path: 大綱文件路徑
        desc_path: 資料欄位描述文件路徑
        output_path: 輸出文件路徑
        model_name: 使用的模型名稱
    
    Returns:
        str: 生成的回應內容,如果失敗則返回 None
    """

    lm = setup_gemini_api(api_key, model_name)

    main_content = read_text_file(main_path)
    description = read_text_file(desc_path)

    chain_prompt = f"""
You are a planning assistant.
Analyze the following outline and column descriptions.

## Outline & Ideas:
{main_content}

## Data Column Descriptions:
{description}

---

Step-by-step:
1. Reflect on the structure and meaning of the content.
2. Formulate relevant and meaningful questions or planning strategies.
3. Be explicit and detailed, use Chain-of-Thought reasoning.
4. Output all thoughts and questions in English only.
"""

    result = lm.basic_request(chain_prompt)

    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(result)
        print(f"✅ Response saved to: {output_path}")
        return result
    except Exception as e:
        print(f"❌ Failed to write output: {e}")
        return None

In [23]:
response = generate_chain_of_thought_response(
    main_path="main.txt",
    desc_path="filtered_data_description.txt",
    output_path="analyze_response.txt"
)

✅ Response saved to: analyze_response.txt


# STEP 3

請LLM根據"analyze_response.txt"思考可以使用的operation並將結果存於 "operations_info.json"

In [37]:
def analyze_operations(analyze_path: str, output_json: str) -> List[str]:
    lm = setup_gemini_api(api_key)
    analysis = read_text_file(analyze_path)

    prompt = f"""
You are a news journalist want to analyze data not forecaster.
Based on the following text analysis, identify multiple useful table operations
and describe the direct meaning of each operation.

## Text Analysis:
{analysis}

---

Please output a numbered list in this format:
1. write: If the table is clear or small enough, generates text based on the tables using the LLM.
2. select_row: Description
3. select_column: Description
4. operation_name: Description
5. operation_name: Description
...

IMPORTANT: operation must contain select_row, select_column, and write in the first three operation.

Give important operations and at most 15 operations.
operation_name should be different and each operation can not be similar.
operation can be apply on many columns is better.
Description just give the original definition of the operation name and give some useful functions name in pandas.
Only include operations and their descriptions. Be concise and clear.
"""

    response = lm.basic_request(prompt)

    operations = []
    operations_dict = {}

    try:
        for line in response.strip().split('\n'):
            if line.strip() == "":
                continue
            if "." in line:
                num, rest = line.split(".", 1)
                if ":" in rest:
                    name, desc = rest.strip().split(":", 1)
                    name = name.strip()
                    desc = desc.strip()
                    operations.append(name)
                    operations_dict[num.strip()] = {"operation": name, "description": desc}

        with open(output_json, 'w', encoding='utf-8') as f:
            json.dump(operations_dict, f, indent=2, ensure_ascii=False)

        print(f"✅ 操作清單與描述已儲存至 {output_json}")
        return operations

    except Exception as e:
        print(f"❌ 回應處理失敗: {e}\n原始回應:\n{response}")
        return []

ops = analyze_operations("analyze_response.txt", "operations_info.json")
print("\n✅ 操作名稱陣列:")
print(ops)

✅ 操作清單與描述已儲存至 operations_info.json

✅ 操作名稱陣列:
['write', 'select_row', 'select_column', 'groupby', 'aggregate', 'sort', 'filter', 'calculate', 'merge', 'join', 'pivot_table', 'rolling_window', 'shift', 'value_counts', 'corr']


# STEP 4

使LLM自動分析table選出合適的operation放入操作池(operations)

In [43]:
class OperationSignature(dspy.Signature):
    """Identify suitable operations for analyzing badminton match data."""
    data_description = dspy.InputField(desc="Overview and sample of the dataset")
    column_descriptions = dspy.InputField(desc="Descriptions of each column in the dataset")
    rules = dspy.InputField(desc="Rules for selecting operations")
    operations_list = dspy.OutputField(desc="A list of suitable operations number (e.g., [1, 2, 3, 4])")

def read_badminton_data(file_path):
    """
    讀取羽球比賽數據 CSV 文件
    
    Args:
        file_path: CSV 文件路徑
    
    Returns:
        pd.DataFrame: 讀取的數據
    """
    try:
        return pd.read_csv(file_path, encoding='utf-8')
    except UnicodeDecodeError:
        return pd.read_csv(file_path, encoding='latin1')


def read_json_file(file_path):
    """
    讀取 JSON 文件
    
    Args:
        file_path: JSON 文件路徑
    
    Returns:
        dict: JSON 數據
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return json.load(file)
    except UnicodeDecodeError:
        with open(file_path, 'r', encoding='latin1') as file:
            return json.load(file)

def parse_column_descriptions(description_text):
    """
    解析欄位描述文本
    
    Args:
        description_text: 欄位描述文本
    
    Returns:
        dict: 欄位名稱到描述的映射
    """
    descriptions = {}
    pattern = r'''
        ^                # Line start
        (\w+)            # Column name
        :\s+             # Colon and space
        (.+?)            # Description text
        (?=\n\w+:\s+|\Z) # Lookahead for next column or end of file
    '''
    matches = re.findall(pattern, description_text, flags=re.M | re.X)
    for col_name, desc in matches:
        clean_desc = ' '.join(desc.split()).strip()
        descriptions[col_name] = clean_desc
    return descriptions

class BadmintonOperationSelector(dspy.Module):
    def __init__(self):
        super().__init__()
        self.chain_of_thought = dspy.ChainOfThought(OperationSignature)

    def forward(self, data_description, column_descriptions, rules):
        result = self.chain_of_thought(
            data_description=data_description,
            column_descriptions=str(column_descriptions),
            rules=str(rules)
        )
        return self.extract_operations_from_result(result.operations_list)

    def extract_operations_from_result(self, operations_text):
        """
        從回應中提取操作編號列表
        支援多種格式:
        - [1, 2, 3, 4]
        - 1, 2, 3, 4
        - 1 2 3 4
        - Operation 1, Operation 2, etc.
        """
        operations = []
        
        # 移除 markdown 代碼塊標記
        operations_text = re.sub(r'```(?:python|json)?\s*', '', operations_text)
        operations_text = operations_text.strip('`').strip()
        
        # 嘗試解析 JSON 格式 [1, 2, 3]
        try:
            # 尋找方括號中的內容
            list_match = re.search(r'\[([^\]]+)\]', operations_text)
            if list_match:
                list_content = list_match.group(1)
                # 提取所有數字
                numbers = re.findall(r'\d+', list_content)
                operations = [int(num) for num in numbers]
                if operations:
                    return operations
        except:
            pass
        
        # 如果沒有方括號,嘗試直接提取所有數字
        numbers = re.findall(r'\d+', operations_text)
        if numbers:
            operations = [int(num) for num in numbers]
            return operations
        
        # 如果以上都失敗,嘗試逐行處理
        lines = operations_text.split('\n')
        for line in lines:
            line = line.strip()
            if not line:
                continue
            # 提取該行中的所有數字
            line_numbers = re.findall(r'\d+', line)
            operations.extend([int(num) for num in line_numbers])
        
        # 去重並排序
        if operations:
            operations = sorted(list(set(operations)))
        
        return operations


def analyze_badminton_match(data_path, column_desc_path, rules_path, model_name="gemini-2.0-flash-exp"):
    """
    分析羽球比賽數據並識別適合的操作
    
    Args:
        data_path: 比賽數據 CSV 文件路徑
        column_desc_path: 欄位描述文件路徑
        rules_path: 操作規則 JSON 文件路徑
        model_name: 使用的模型名稱
    
    Returns:
        list: 識別出的操作編號列表 (整數)
    """
    
    print("Reading badminton match data...")
    try:
        match_data = read_badminton_data(data_path)
        columns_desc_content = read_text_file(column_desc_path)
        rules = read_json_file(rules_path)
    except Exception as e:
        print(f"❌ Error reading files: {e}")
        return []

    column_descriptions = parse_column_descriptions(columns_desc_content)
    setup_gemini_api(api_key, model_name)

    data_sample = match_data.to_string()
    data_description = f"""
    one match data:
    {data_sample}

    Data shape: {match_data.shape[0]} rows, {match_data.shape[1]} columns
    Columns: {', '.join(match_data.columns)}
    """

    selector = BadmintonOperationSelector()
    operations = selector.forward(data_description, column_descriptions, rules)

    print(f"✅ Identified {len(operations)} suitable operations:")
    for i, op in enumerate(operations, 1):
        print(f"{i}. Operation {op}")

    return operations

In [44]:
operations = analyze_badminton_match(
    data_path="set1.csv",
    column_desc_path="data_description.txt",
    rules_path="operations_info.json"
)
print("\nFinal operations array:", operations)

Reading badminton match data...
✅ Identified 6 suitable operations:
1. Operation 14
2. Operation 4
3. Operation 5
4. Operation 15
5. Operation 6
6. Operation 8

Final operations array: [14, 4, 5, 15, 6, 8]


將所挑選出來的操作寫入"operations.json"

In [45]:
import json

# 從 JSON 檔案讀取 operations
original_operations_dict = read_json_file("operations_info.json")

# 你想要挑選的 operation 編號（根據實際需求修改這個 list）
selected_numbers = operations

# 根據 selected_numbers 選出對應操作，並從 1 開始重新編號
filtered_operations = []
for new_number, original_number in enumerate(selected_numbers, start=1):
    # 將數字轉換為字串鍵來查找
    key = str(original_number)
    if key in original_operations_dict:
        op_data = original_operations_dict[key]
        filtered_operations.append({
            "number": new_number,
            "operation": op_data["operation"],
            "description": op_data["description"]
        })
    else:
        print(f"⚠️ Warning: Operation {original_number} not found in operations_info.json")

# 新的 JSON 結構
output_json = {
    "description": "Selected operations for badminton data analysis.",
    "requirements": [
        "The output must be based on the input data; do not hallucinate.",
        "Give me the list of numbers."
    ],
    "operations": filtered_operations
}

# 寫入 JSON 檔案
with open("operations.json", "w", encoding="utf-8") as f:
    json.dump(output_json, f, ensure_ascii=False, indent=2)

print(f"✅ operations.json has been created with {len(filtered_operations)} operations.")
print(f"Selected operations: {selected_numbers}")

✅ operations.json has been created with 6 operations.
Selected operations: [14, 4, 5, 15, 6, 8]


# STEP 5

篩選出最合適的1/2 operations

In [62]:
import os
import re
import json

def load_operations_from_json(json_file_path):
    """
    Load operations from JSON file
    支援兩種格式:
    1. 舊格式: {"1": {"operation": "...", "description": "..."}, ...}
    2. 新格式: {"operations": [{"number": 1, "name": "...", "description": "..."}, ...]}
    """
    try:
        data = read_json_file(json_file_path)
        
        operations_data = []
        
        # 檢查是新格式還是舊格式
        if 'operations' in data and isinstance(data['operations'], list):
            # 新格式
            operations_data = data['operations']
        else:
            # 舊格式: 數字字串作為鍵
            for key in sorted(data.keys(), key=lambda x: int(x) if x.isdigit() else 0):
                if key.isdigit():
                    op_data = data[key]
                    operations_data.append({
                        'number': int(key),
                        'name': op_data.get('operation', ''),
                        'description': op_data.get('description', '')
                    })
        
        # Create formatted operation strings for LLM processing
        operation_strings = []
        operation_details = []
        
        for op in operations_data:
            number = op.get('number', '')
            name = op.get('name', '')
            description = op.get('description', '')
            
            # Format as: "number. name: description"
            if number and name and description:
                formatted_op = f"{number}. {name}: {description}"
                operation_strings.append(formatted_op)
                operation_details.append({
                    'number': number,
                    'name': name,
                    'description': description,
                    'formatted': formatted_op
                })
        
        print(f"從 {json_file_path} 成功載入 {len(operation_strings)} 個操作")
        return operation_details, operation_strings
        
    except FileNotFoundError:
        print(f"錯誤: 找不到文件 {json_file_path}")
        return [], []
    except json.JSONDecodeError:
        print(f"錯誤: {json_file_path} 不是有效的 JSON 文件")
        return [], []
    except Exception as e:
        print(f"載入操作時發生錯誤: {e}")
        return [], []


def filter_operations_direct_gemini(api_key, operations_list, operation_details, data_sample, data_info, removal_percentage=0.25, model_name="gemini-2.0-flash"):
    """
    Use Gemini API directly to filter operations and return operation numbers
    """
    gemini_lm = GeminiOpenAI(api_key=api_key, model_name=model_name)
    
    operations_count = len(operations_list)
    operations_to_remove = int(operations_count * removal_percentage)
    operations_to_keep = operations_count - operations_to_remove
    
    # Create numbered list of operations for easier reference
    numbered_operations = "\n".join([f"{i+1}. {op}" for i, op in enumerate(operations_list)])
    
    prompt = f"""
我有一個羽球比賽的資料集和 {operations_count} 個分析操作。

資料樣本:
{data_sample}

資料集資訊:
{data_info}

操作清單:
{numbered_operations}

請幫我分析並移除 {operations_to_remove} 個最不合適的操作（約 {removal_percentage*100:.0f}%），保留 {operations_to_keep} 個最適合的操作。

請考慮以下標準來決定移除哪些操作：
1. 與實際資料欄位的相關性
2. 在給定資料集結構下的可行性
3. 對羽球比賽分析的實用價值
4. 避免重複或過於相似的操作

請先說明你的分析思路，然後**只提供要保留操作的編號**（從操作描述開頭提取的編號）。

請用以下格式回答：

分析思路：
[你的分析]

保留的操作編號：
[編號1, 編號2, 編號3, ...]
"""
    
    response = gemini_lm.basic_request(prompt)
    
    # Extract kept operation numbers from the response
    kept_operation_numbers = extract_operation_numbers_from_response(response, operation_details)
    
    return kept_operation_numbers, response


def extract_operation_numbers_from_response(response, operation_details):
    """
    Extract the operation numbers to keep from Gemini's response
    """
    kept_numbers = []
    
    # Look for the section with kept operation numbers
    lines = response.split('\n')
    
    # Find the start of the operations list
    start_extracting = False
    for line in lines:
        line = line.strip()
        
        # Look for section headers
        if any(keyword in line.lower() for keyword in ['保留的操作編號', '保留操作編號', '編號', 'numbers']):
            start_extracting = True
            # 檢查標題行本身是否包含數字
            bracket_match = re.search(r'\[(.*?)\]', line)
            if bracket_match:
                numbers_text = bracket_match.group(1)
                for item in numbers_text.split(','):
                    number = re.search(r'(\d+)', item.strip())
                    if number:
                        kept_numbers.append(int(number.group(1)))
                if kept_numbers:
                    break
            continue
        
        if start_extracting and line:
            # Try to extract numbers from various formats
            bracket_match = re.search(r'\[(.*?)\]', line)
            if bracket_match:
                numbers_text = bracket_match.group(1)
                for item in numbers_text.split(','):
                    number = re.search(r'(\d+)', item.strip())
                    if number:
                        kept_numbers.append(int(number.group(1)))
                break
            
            # Format 2: Numbered list or comma-separated numbers
            numbers = re.findall(r'\b(\d+)\b', line)
            if numbers:
                kept_numbers.extend([int(n) for n in numbers])
                break
    
    # Remove duplicates and validate against available operations
    valid_numbers = []
    available_numbers = [detail['number'] for detail in operation_details]
    
    for num in kept_numbers:
        if num in available_numbers and num not in valid_numbers:
            valid_numbers.append(num)
    
    return valid_numbers


def get_data_summary(dataframe):
    """
    Generate a comprehensive summary of the dataset
    """
    summary = f"""
資料集概要:
- 總行數: {dataframe.shape[0]}
- 總列數: {dataframe.shape[1]}
- 欄位名稱: {', '.join(dataframe.columns)}

各欄位資訊:
"""
    
    for col in dataframe.columns:
        col_info = f"  - {col}: "
        if dataframe[col].dtype in ['object', 'string']:
            unique_values = dataframe[col].unique()[:10]
            col_info += f"類別型資料, 獨特值範例: {', '.join(map(str, unique_values))}"
        else:
            col_info += f"數值型資料, 範圍: {dataframe[col].min()} - {dataframe[col].max()}"
        
        summary += col_info + "\n"
    
    return summary


def filter_badminton_operations(operations_list, operation_details, dataframe, removal_percentage=0.25, model_name="gemini-2.0-flash"):
    """
    Main function to filter operations using Gemini LLM and return operation numbers
    (只保留方法二: 直接 Gemini API)
    """
    api_key = os.getenv("Gemini_API")
    
    print(f"原始操作數量: {len(operations_list)}")
    print("原始操作清單:")
    for i, op in enumerate(operations_list, 1):
        print(f"  {i}. {op}")
    
    # Get data summary
    data_summary = get_data_summary(dataframe)
    data_sample = dataframe.head(5).to_string()
    
    print(f"\n使用 Gemini LLM 過濾操作 (移除 {removal_percentage*100:.0f}%)...")
    
    # Method 2: Direct Gemini API call
    try:
        print("方法: 直接使用 Gemini API...")
        direct_filtered_numbers, gemini_response = filter_operations_direct_gemini(
            api_key, 
            operations_list, 
            operation_details,
            data_sample, 
            data_summary, 
            removal_percentage,
            model_name
        )
        print(f"直接 API 方法保留了 {len(direct_filtered_numbers)} 個操作編號")
        
        print("\nGemini 回應:")
        print("="*50)
        print(gemini_response)
        print("="*50)
        
    except Exception as e:
        print(f"直接 API 方法失敗: {e}")
        direct_filtered_numbers = []
    
    # Final results
    if direct_filtered_numbers:
        final_operation_numbers = direct_filtered_numbers
        print(f"\n使用直接 API 方法的結果")
    else:
        # Fallback
        target_count = int(len(operations_list) * (1 - removal_percentage))
        final_operation_numbers = [detail['number'] for detail in operation_details[:target_count]]
        print(f"\n直接 API 方法失敗，使用前 {target_count} 個操作編號作為備案")
    
    print(f"\n最終保留的操作編號 ({len(final_operation_numbers)} 個):")
    for i, number in enumerate(final_operation_numbers, 1):
        for detail in operation_details:
            if detail['number'] == number:
                print(f"  {i}. 編號 {number}: {detail['name']}")
                break
    
    return final_operation_numbers


def create_filtered_operations_json(original_json_path, filtered_numbers, output_path="filtered_operations.json"):
    """
    創建過濾後的操作 JSON 文件
    """
    # 讀取原始 JSON
    all_data = read_json_file(original_json_path)
    
    # 處理兩種格式
    if 'operations' in all_data and isinstance(all_data['operations'], list):
        # 新格式
        original_operations = all_data['operations']
    else:
        # 舊格式: 轉換為新格式
        original_operations = []
        for key in sorted(all_data.keys(), key=lambda x: int(x) if x.isdigit() else 0):
            if key.isdigit():
                op_data = all_data[key]
                original_operations.append({
                    'number': int(key),
                    'name': op_data.get('operation', ''),
                    'description': op_data.get('description', '')
                })
    
    # 根據 filtered_numbers 選出對應操作，並從 1 開始重新編號
    filtered_operations = []
    for new_number, original_number in enumerate(filtered_numbers, start=1):
        for op in original_operations:
            if op["number"] == original_number:
                # 使用 'name' 或 'operation' 欄位
                op_name = op.get('name') or op.get('operation', '')
                filtered_operations.append({
                    "number": new_number,
                    "operation": op_name,
                    "description": op["description"]
                })
                break
    
    # 新的 JSON 結構
    output_json = {
        "description": "Selected operations for badminton data analysis.",
        "requirements": [
            "The output must be based on the input data; do not hallucinate.",
            "Give me the list of numbers."
        ],
        "operations": filtered_operations
    }
    
    # 寫入 JSON 檔案
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(output_json, f, ensure_ascii=False, indent=2)
    
    print(f"✅ {output_path} has been created with {len(filtered_operations)} operations.")
    return filtered_operations



In [63]:
# Load operations from JSON file
json_file_path = "operations_info.json"  # 支援舊格式
operation_details, operation_strings = load_operations_from_json(json_file_path)

# Load badminton data
example_df = read_badminton_data('set1.csv')

# Filter operations and get operation numbers
filtered_operation_numbers = filter_badminton_operations(
    operation_strings,
    operation_details,
    example_df, 
    removal_percentage=0.2
)

print(f"\n保留的操作編號清單: {filtered_operation_numbers}")

# 創建過濾後的 JSON 文件
create_filtered_operations_json(
    "operations.json",
    filtered_operation_numbers,
    "filtered_operations.json"
)

從 operations_info.json 成功載入 15 個操作
原始操作數量: 15
原始操作清單:
  1. 1. write: Generate text based on the table after other operations, providing insights and conclusions. This operation transforms data into a readable news story. Useful functions: N/A
  2. 2. select_row: Select a subset of rows based on specific criteria (e.g., score range, player, rally length). This allows focusing on particular game situations. Useful functions: `df.loc[]`, `df.iloc[]`
  3. 3. select_column: Select specific columns relevant to the analysis (e.g., `type`, `time`, `roundscore_A`, `roundscore_B`). Useful functions: `df[['column1', 'column2']]`
  4. 4. groupby: Group rows based on one or more columns (e.g., `player`, `type`, `lose_reason`) to calculate aggregate statistics. This helps identify trends and patterns. Useful functions: `df.groupby()`
  5. 5. aggregate: Calculate summary statistics (e.g., mean, median, standard deviation, count) for grouped data. This quantifies trends and differences. Useful functio

[{'number': 1,
  'operation': 'value_counts',
  'description': "Count the occurrences of each unique value in a column. Useful functions: `df['column'].value_counts()`"},
 {'number': 2,
  'operation': 'groupby',
  'description': 'Group rows based on one or more columns (e.g., `player`, `type`, `lose_reason`) to calculate aggregate statistics. This helps identify trends and patterns. Useful functions: `df.groupby()`'},
 {'number': 3,
  'operation': 'aggregate',
  'description': 'Calculate summary statistics (e.g., mean, median, standard deviation, count) for grouped data. This quantifies trends and differences. Useful functions: `df.agg()`'},
 {'number': 4,
  'operation': 'corr',
  'description': 'Calculate the correlation between columns. Useful functions: `df.corr()`'},
 {'number': 5,
  'operation': 'sort',
  'description': 'Sort the data by one or more columns (e.g., `time`, `rally`, `roundscore_A`) to identify trends and patterns over time or score progression. Useful functions: `df

# STEP 6

根據真實table只保留重要70%操作，保留'write' 'select_col' 'select_row'三個重要操作，到'selected_operations.json'

操作提取已完成!!

In [ ]:
# 正式版 - 使用 Gemini 過濾羽球比賽分析操作
import pandas as pd
import numpy as np
import dspy
import re
import json
import os
from openai import OpenAI
import time


def get_data_summary(dataframe):
    """
    Generate a comprehensive summary of the dataset
    """
    summary = f"資料集概要:\n- 總行數: {dataframe.shape[0]}\n- 總列數: {dataframe.shape[1]}\n- 欄位名稱: {', '.join(dataframe.columns)}\n\n各欄位資訊:\n"
    for col in dataframe.columns:
        summary += f"  - {col}: "
        if dataframe[col].dtype in ['object', 'string']:
            summary += f"類別型資料, 獨特值範例: {', '.join(map(str, dataframe[col].unique()[:10]))}\n"
        else:
            summary += f"數值型資料, 範圍: {dataframe[col].min()} - {dataframe[col].max()}\n"
    return summary

def extract_operation_numbers_from_response(response):
    """
    從回應中提取操作編號列表
    支援多種格式
    """
    # 方法1: 匹配代碼塊中的數組
    pattern1 = r'```\s*\[([\d,\s]+)\]\s*```'
    match = re.search(pattern1, response)
    
    if match:
        array_str = match.group(1)
        operation_list = [int(num) for num in array_str.replace(' ', '').split(',')]
        print(f"提取到操作列表: {operation_list}")
        return operation_list
    
    # 方法2: 匹配普通方括號中的數組
    pattern2 = r'\[([\d,\s]+)\]'
    match = re.search(pattern2, response)
    
    if match:
        array_str = match.group(1)
        operation_list = [int(num) for num in array_str.replace(' ', '').split(',')]
        print(f"提取到操作列表: {operation_list}")
        return operation_list
    
    # 方法3: 提取所有數字
    numbers = re.findall(r'\b(\d+)\b', response)
    if numbers:
        operation_list = [int(num) for num in numbers]
        print(f"提取到操作列表: {operation_list}")
        return operation_list
    
    print("⚠️ 未找到排序數組")
    return []

def filter_badminton_operations(operation_details, operation_strings, df, api_key, outline_path='outline.txt', model_name="gemini-2.0-flash", max_retries=3):
    """
    使用 Gemini 根據重要性排序操作
    
    Args:
        operation_details: 操作詳細資訊列表
        operation_strings: 操作格式化字串列表
        df: 數據框
        api_key: API 金鑰
        outline_path: 大綱文件路徑
        model_name: 模型名稱
        max_retries: 最大重試次數
    
    Returns:
        tuple: (排序後的操作編號列表, 完整回應)
    """
    gemini = GeminiOpenAI(api_key=api_key, model_name=model_name)
    data_summary = get_data_summary(df)
    
    # 限制資料樣本大小
    data_sample = df.head(10).to_string()
    if len(data_sample) > 3000:
        data_sample = data_sample[:3000] + "...\n[資料已截斷]"
    
    outline = read_text_file(outline_path)
    
    print(f"操作數量: {len(operation_strings)}")
    
    prompt = f"""
我有一個撰寫新聞的大綱與比賽的資料集和 {len(operation_strings)} 個分析操作，請依據操作重要性排序(由高到低)。

大綱:
{outline}

資料樣本:
{data_sample}

資料集資訊:
{data_summary}

操作清單:
{chr(10).join(operation_strings)}

請先根據 chain-of-thought 分析，然後將操作編號根據重要性排序，每個編號僅在陣列中出現一次，陣列長度應為 {len(operation_strings)}。

最後請以以下格式輸出排序結果:[1, 2, 3, ...]"""
    
    response = gemini.basic_request(prompt, max_retries=max_retries)
    
    # 檢查是否為錯誤回應
    if "⚠️" in response:
        print(f"❌ API 回應錯誤")
        return [], response
    
    return extract_operation_numbers_from_response(response), response

def create_selected_operations_json(operation_details, sorted_numbers, keep_percentage=0.7, force_include=[1, 2, 3], output_path="selected_operations.json"):
    """
    創建選擇的操作 JSON 文件
    
    Args:
        operation_details: 操作詳細資訊列表
        sorted_numbers: 排序後的操作編號列表
        keep_percentage: 保留比例
        force_include: 強制包含的操作編號
        output_path: 輸出文件路徑
    
    Returns:
        list: 選擇的操作列表
    """
    # 計算要保留的操作數量
    keep_count = int(keep_percentage * len(sorted_numbers))
    
    # 選擇前 N 個操作
    selected_numbers = sorted_numbers[:keep_count]
    
    # 確保強制包含的操作在列表中
    selected_numbers = list(set(selected_numbers) | set(force_include))
    
    print(f"選擇了 {len(selected_numbers)} 個操作 (保留比例: {keep_percentage*100:.0f}%)")
    print(f"選擇的操作編號: {selected_numbers}")
    
    # 創建新的操作列表
    new_operations = []
    for new_id, num in enumerate(selected_numbers, 1):
        for detail in operation_details:
            if int(detail['number']) == int(num):
                new_operations.append({
                    'number': new_id,
                    'operation': detail['operation'],
                    'description': detail['description']
                })
                break
    
    # 寫入 JSON 文件
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(new_operations, f, indent=2, ensure_ascii=False)
    
    print(f"✅ {output_path} has been created with {len(new_operations)} operations.")
    return new_operations

def read_badminton_data(file_path):
    """
    讀取羽球比賽數據 CSV 文件
    """
    try:
        return pd.read_csv(file_path, encoding='utf-8')
    except UnicodeDecodeError:
        return pd.read_csv(file_path, encoding='latin1')

In [68]:
# 載入操作
json_file_path = "operations_info.json"
operation_details, operation_strings = load_operations_from_json(json_file_path)

# 載入數據
df = read_badminton_data("filtered_set1.csv")

# 獲取 API 金鑰
api_key = os.getenv("Gemini_API") 
# 排序操作
sorted_numbers, response = filter_badminton_operations(
    operation_details, 
    operation_strings, 
    df, 
    api_key, 
    outline_path='main.txt'
)

print(f"完整回應:\n{response}\n")
print(f"排序後的操作編號: {sorted_numbers}")

# 創建選擇的操作 JSON
selected_ops = create_selected_operations_json(
    operation_details,
    sorted_numbers,
    keep_percentage=0.7,
    force_include=[1, 2, 3],
    output_path="selected_operations.json"
)

從 operations_info.json 成功載入 15 個操作
操作數量: 15
提取到操作列表: [1, 3, 4, 5, 14, 8, 6, 2, 7, 11, 12, 13, 15, 9, 10]
完整回應:
Let's analyze the importance of each operation for generating badminton news from the given dataset. The goal is to extract insights that are newsworthy and insightful for badminton fans, while adhering to the restriction of avoiding common sense observations.

Here's a chain of thought process:

* **Essential for initial overview and context:**
    * **1. Observe the outcome of the game and the final score:** This is the most basic information. Understanding who won and by how much is fundamental to any news report. Thus, calculating final scores is vital.
    * **2. The duration of the entire competition:** The length of the match puts the result into context. A long, grueling match tells a different story than a quick victory.  So, total time is vital.

* **Core for understanding game dynamics:**
    * **Analyzing point scoring patterns and trends:** How were points scored?

In [69]:
import json

# 讀取 filtered_operations.json
with open('filtered_operations.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 取得目前最大 number
existing_numbers = [op['number'] for op in data['operations']]
max_number = max(existing_numbers) if existing_numbers else 0

# 新增的 operations (注意使用 'operation' 而非 'name')
new_operations = [
    {
        "number": max_number + 1,
        "operation": "select_row",
        "description": "Selects rows based on their row indices."
    },
    {
        "number": max_number + 2,
        "operation": "select_col",
        "description": "Selects columns based on their column names."
    },
    {
        "number": max_number + 3,
        "operation": "write",
        "description": "If the table is small enough, generates text based on the tables using the LLM; represents the leaf node of the tree."
    }
]

# 將新操作加入原始資料
data['operations'].extend(new_operations)

# 寫回 JSON 檔
with open('selected_operations.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

# 輸出所有操作的 number 列表
all_numbers = [op['number'] for op in data['operations']]
print(f"所有操作編號: {all_numbers}")
print(f"總操作數量: {len(all_numbers)}")

所有操作編號: [1, 2, 3, 4, 5, 6, 7, 8, 9]
總操作數量: 9


# STEP 7

根據Table,得到要執行的操作與參數

In [74]:
import pandas as pd
import json
import os

class ContentPlanner:
    def __init__(self, api_key, model_name="gemini-2.0-flash"):
        self.api_key = api_key
        self.model = GeminiOpenAI(api_key=api_key, model_name=model_name)
        
    def generate_operations(self, tables, table_description, operation_description, 
                          operation_history, operation_pool, max_depth=5, max_degree=3, outline_path='main.txt'):
        """
        使用Gemini生成operations和arguments
        """
        
        # 構建完整的提示詞
        prompt = f"""System : You are a content planner for the report. Please follow the outline. Please select candidate Operations and corresponding Arguments from the Operation Pool based on the input Tables and Operation History. These candidate Operations will be the next Operation in the Operation History .

# Requirements
1. Strictly adhere to the requirements .
2. The output must be in English .
3. The output must be based on the input data ; do not hallucinate .
4. The length of Operation History must be less than or equal to {max_depth}.
5. The number of Operations must be less than or equal to {max_degree}.
6. Only select Operations from the Operation Pool .
7. Arguments must match the format required by the corresponding Operations .
8. Operations & Arguments must follow this format : [ operation_1 ( argument_1 , ...) , operation_2 ( argument_2 , ...) , operation_3 ( argument_3 , ...) , ...]
9. Only output Operations & Arguments !
10. If Table is big or Level is low, it should be more Operations include select_col or select_row not write.
11. If the length of Operation History is short, then more operations or more arguments.
12. Write operations do not need argument.

#outline
{read_text_file(outline_path)}

# Table Description
{table_description}

# Operation Description
{json.dumps(operation_description, indent=2, ensure_ascii=False)}

User : # Test
## Tables
{tables}

## Operation History
{operation_history}

## Operation Pool
{operation_pool}

## Operations & Arguments"""

        try:
            print("正在向Gemini發送請求...")
            response = self.model.basic_request(prompt)
            
            if response and "⚠️" not in response:
                print("成功獲得Gemini回應")
                return response.strip()
            else:
                print("Gemini回應為空或出現錯誤")
                return None
                
        except Exception as e:
            print(f"Gemini API請求失敗: {e}")
            return None

def run_content_planner(csv_path='filtered_set1.csv', 
                       table_desc_path='filtered_data_description.txt',
                       operations_path='selected_operations.json',
                       outline_path='analyze_response.txt',
                       max_depth=5,
                       max_degree=5):
    """
    運行內容規劃器
    
    Args:
        csv_path: CSV 數據文件路徑
        table_desc_path: 表格描述文件路徑
        operations_path: 操作描述 JSON 文件路徑
        outline_path: 大綱文件路徑
        max_depth: 最大深度
        max_degree: 最大分支度
    
    Returns:
        tuple: (操作和參數字串, 更新後的操作歷史, 當前層級)
    """
    # 設置API密鑰
    api_key = os.getenv("Gemini_API")
    if not api_key:
        print("❌ Gemini_API 環境變數未設定")
        return None, None, None
    
    print("Content Planner for Badminton Game Report")
    print("="*50)
    
    print("正在載入數據...")
    
    # 讀取CSV檔案
    TABLES = pd.read_csv(csv_path)
    tables_str = TABLES.head(10).to_string()  # 限制顯示前10行
    if len(tables_str) > 3000:
        tables_str = tables_str[:3000] + "...\n[資料已截斷]"
    print(f"成功載入CSV: {TABLES.shape[0]} 行, {TABLES.shape[1]} 列")
    
    # 讀取表格描述
    TABLE_DESCRIPTION = read_text_file(table_desc_path)
    if not TABLE_DESCRIPTION:
        TABLE_DESCRIPTION = "No table description available"
    print(f"載入表格描述: {len(TABLE_DESCRIPTION)} 字符")
    
    # 讀取操作描述
    OPERATION_DESCRIPTION = read_json_file(operations_path)
    print(f"載入操作描述 JSON")
    
    # 設置其他變數
    OPERATION_HISTORY = ['root(None)']
    Level = 0
    
    # 從操作描述中提取操作池
    # 處理新格式: 直接是操作列表
    if isinstance(OPERATION_DESCRIPTION, list):
        OPERATION_POOL = [op.get('operation', op.get('name', '')) for op in OPERATION_DESCRIPTION if op.get('operation') or op.get('name')]
    # 處理舊格式: {"operations": [...]}
    elif isinstance(OPERATION_DESCRIPTION, dict) and 'operations' in OPERATION_DESCRIPTION:
        OPERATION_POOL = [op.get('operation', op.get('name', '')) for op in OPERATION_DESCRIPTION['operations'] if op.get('operation') or op.get('name')]
    else:
        print("❌ 無法識別的 JSON 格式")
        return None, None, None
    
    print(f"操作池 ({len(OPERATION_POOL)} 個): {OPERATION_POOL}")
    print(f"操作歷史: {OPERATION_HISTORY}")
    
    # 初始化內容規劃器
    planner = ContentPlanner(api_key)
    
    # 生成操作和參數
    print("\n開始生成操作和參數...")
    operations_and_arguments = planner.generate_operations(
        tables=tables_str,
        table_description=TABLE_DESCRIPTION,
        operation_description=OPERATION_DESCRIPTION,
        operation_history=OPERATION_HISTORY,
        operation_pool=OPERATION_POOL,
        max_depth=max_depth,
        max_degree=max_degree,
        outline_path=outline_path
    )
    
    # 更新操作歷史
    if operations_and_arguments:
        OPERATION_HISTORY.append(operations_and_arguments)
        Level += 1
        
        print("\n" + "="*50)
        print("GEMINI 輸出結果:")
        print("="*50)
        print(operations_and_arguments)
        print("="*50)
        print(f"當前層級: {Level}")
        print(f"更新後的操作歷史: {OPERATION_HISTORY}")
        
        return operations_and_arguments, OPERATION_HISTORY, Level
    else:
        print("❌ 未能生成操作和參數")
        return None, OPERATION_HISTORY, Level

In [ ]:
# 或指定自定義參數
operations_result, history, level = run_content_planner(
    csv_path='filtered_set1.csv',
    table_desc_path='filtered_data_description.txt',
    operations_path='selected_operations.json',
    outline_path='analyze_response.txt',
    max_depth=5,
    max_degree=5
)

if operations_result:
    print(f"\n生成的操作: {operations_result}")
    print(f"操作歷史長度: {len(history)}")
    print(f"當前層級: {level}")

Content Planner for Badminton Game Report
正在載入數據...
成功載入CSV: 315 行, 9 列
載入表格描述: 481 字符
載入操作描述 JSON
操作池 (9 個): ['value_counts', 'groupby', 'aggregate', 'corr', 'sort', 'calculate', 'select_row', 'select_col', 'write']
操作歷史: ['root(None)']

開始生成操作和參數...
正在向Gemini發送請求...
嘗試 1/3 失敗: Error code: 503 - [{'error': {'code': 503, 'message': 'The service is currently unavailable.', 'status': 'UNAVAILABLE'}}]
服務暫時不可用，等待 2 秒後重試...
嘗試 2/3 失敗: Error code: 503 - [{'error': {'code': 503, 'message': 'The service is currently unavailable.', 'status': 'UNAVAILABLE'}}]
服務暫時不可用，等待 4 秒後重試...
成功獲得Gemini回應

GEMINI 輸出結果:
[select_col ( type , player , getpoint_player )]
當前層級: 1
更新後的操作歷史: ['root(None)', '[select_col ( type , player , getpoint_player )]']
Content Planner for Badminton Game Report
正在載入數據...
成功載入CSV: 315 行, 9 列
載入表格描述: 481 字符
載入操作描述 JSON
操作池 (9 個): ['value_counts', 'groupby', 'aggregate', 'corr', 'sort', 'calculate', 'select_row', 'select_col', 'write']
操作歷史: ['root(None)']

開始生成操作和參數...
正在向Gemini發

解析LLM response內容

In [76]:
# 提取方括号内的内容
start = operations_result.find('[') + 1
end = operations_result.rfind(']')
content = operations_result[start:end].strip()

elements = []
current = []
stack = 0

# 遍历字符进行解析
for char in content:
    if char == '(':
        stack += 1
        current.append(char)
    elif char == ')':
        stack -= 1
        current.append(char)
    elif char == ',' and stack == 0:
        elements.append(''.join(current).strip())
        current = []
    else:
        current.append(char)

# 添加最后一个元素
if current:
    elements.append(''.join(current).strip())

print(elements)

['select_col ( player, type, lose_reason, getpoint_player )', 'value_counts ( player )', 'value_counts ( type )', 'value_counts ( lose_reason )', 'value_counts ( getpoint_player )']


# STEP 8

根據欄位型態與'operation_name' 和 'operation_argument'，請LLM撰寫可以執行的操作程式碼

取欄位型態

In [77]:
import pandas as pd
df_copy = pd.read_csv("filtered_set1.csv")
df = df_copy
print(df.dtypes)


Unnamed: 0          int64
rally               int64
time               object
roundscore_A        int64
roundscore_B        int64
player             object
type               object
lose_reason        object
getpoint_player    object
dtype: object


In [78]:
import re
import pandas as pd

class DataFrameOperator:
    def __init__(self, api_key):
        self.lm = setup_gemini_api(api_key)

    def generate_code(self, operation, df_info, df_path):
        prompt = f"""
        你是一個專業的Python資料分析助手。欄位名稱以資料欄位類型提供為主，根據以下要求生成操作DataFrame的程式碼：

        要執行的操作: {operation}

        CSV數據集: {df_path}

        資料欄位類型:
        {df_info}

        生成要求：
        讀取CSV數據集，並存入DataFrame後，使用要執行的操作後，將修改後的DataFrame存入'tmp.csv'，撰寫完整python code.
        切忌每個操作參數都需要使用

        輸出格式：
        ```python
        # 你的程式碼
        ```
        """
        return self.lm.basic_request(prompt)

    def safe_execute(self, code, df):
        try:
            code_block = re.search(r'```python\n(.*?)\n```', code, re.DOTALL)
            if code_block:
                code = code_block.group(1)

            # 寫入暫存 CSV 檔案作為模擬 df.csv 路徑
            df.to_csv("input_tmp.csv", index=False)

            # 建立安全執行環境
            exec_globals = {'pd': pd}
            exec_locals = {}

            # 執行生成的程式碼
            exec(code, exec_globals, exec_locals)

            # 從 tmp.csv 讀取處理後的結果
            result_df = pd.read_csv("tmp.csv")
            return result_df

        except Exception as e:
            print(f"執行錯誤: {str(e)}")
            return df



# 初始化
API_KEY = os.getenv("Gemini_API")
operator = DataFrameOperator(API_KEY)

# 獲取資料資訊
df_info = df.info()
df_path = "filtered_set1.csv"
operation_def = elements[0]
#print(operation)

generated_code = operator.generate_code(
    operation=operation_def,
    df_info=df_info,
    df_path=df_path
)

print("生成的程式碼：")
print(generated_code)

# 執行操作
processed_df = operator.safe_execute(generated_code, df)

print("\n處理結果：")
print(processed_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315 entries, 0 to 314
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       315 non-null    int64 
 1   rally            315 non-null    int64 
 2   time             315 non-null    object
 3   roundscore_A     315 non-null    int64 
 4   roundscore_B     315 non-null    int64 
 5   player           315 non-null    object
 6   type             315 non-null    object
 7   lose_reason      36 non-null     object
 8   getpoint_player  36 non-null     object
dtypes: int64(4), object(5)
memory usage: 22.3+ KB
生成的程式碼：
```python
import pandas as pd

# 讀取CSV數據集
df = pd.read_csv('filtered_set1.csv')

# 要執行的操作：select_col ( player, type, lose_reason, getpoint_player )
# 假設 select_col 的功能是選擇指定的欄位

selected_columns = ['player', 'type', 'lose_reason', 'getpoint_player']
df_selected = df[selected_columns]

# 將修改後的DataFrame存入 'tmp.csv'
df_selected.to_csv('tmp.csv'

In [ ]:
import random
#可刪
class TreeNode:
    def __init__(self, value, level=0, text="", table=None):
        self.value = value
        self.children = []
        self.level = level
        self.text = text
        self.table = table
        
    def __repr__(self):
        return f"TreeNode({self.value}, level={self.level})"

def build_random_tree(current_depth=1, max_depth=5, max_degree=5, value_counter=None):
    if value_counter is None:
        value_counter = [0]
    
    value_counter[0] += 1
    node = TreeNode(value_counter[0], level=current_depth)

    if current_depth >= max_depth or random.random() < 0.3:
        return node  # 葉節點

    degree = random.randint(1, max_degree)
    print(f"Level {current_depth}, Node {value_counter[0]}: {degree} children")
    
    for _ in range(degree):
        child = build_random_tree(current_depth + 1, max_depth, max_degree, value_counter)
        node.children.append(child)

    return node

def print_tree(node, level=0):
    print("  " * level + f"- Node({node.value}, Level={node.level})")
    for child in node.children:
        print_tree(child, level + 1)

# 建立並印出隨機樹
random.seed(42)  # 可重現性
root = build_random_tree()
print("\n樹狀結構:")
print_tree(root)

Level 1, Node 1: 1 children
Level 2, Node 2: 2 children
Level 3, Node 4: 5 children
Level 4, Node 6: 1 children
Level 4, Node 10: 5 children

樹狀結構:
- Node(1, Level=1)
  - Node(2, Level=2)
    - Node(3, Level=3)
    - Node(4, Level=3)
      - Node(5, Level=4)
      - Node(6, Level=4)
        - Node(7, Level=5)
      - Node(8, Level=4)
      - Node(9, Level=4)
      - Node(10, Level=4)
        - Node(11, Level=5)
        - Node(12, Level=5)
        - Node(13, Level=5)
        - Node(14, Level=5)
        - Node(15, Level=5)


# STEP final

In [ ]:
import pandas as pd
import json
import google.generativeai as genai
import os
import dspy
import ast
import re
from typing import List, Dict, Any, Optional, Set
import copy
import hashlib
import logging
from datetime import datetime
import sys
import builtins
# 設置日誌
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# ===== 基於參考程式碼的函數 =====
def read_text_file(file_path):
    """讀取文本文件"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        return "No file available"
    except Exception as e:
        logger.error(f"讀取文件錯誤: {e}")
        return "Error reading file"

def read_json_file(file_path):
    """讀取JSON文件"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return json.load(file)
    except UnicodeDecodeError:
        with open(file_path, 'r', encoding='latin1') as file:
            return json.load(file)
    except FileNotFoundError:
        # 返回默認操作集合
        return [
            {"name": "select_column", "description": "選擇特定欄位"},
            {"name": "value_counts", "description": "計算值的頻次"},
            {"name": "groupby", "description": "按欄位分組"},
            {"name": "sort_values", "description": "排序數據"},
            {"name": "filter_rows", "description": "過濾行數據"},
            {"name": "write", "description": "撰寫分析文本"}
        ]

# ===== 改進的樹節點類別 =====
class TreeNode:
    """改進的樹節點類別，增加語意驗證和追蹤功能"""
    def __init__(self, level: int = 0, text: str = "", table: pd.DataFrame = None, operation: str = None):
        self.children: List['TreeNode'] = []
        self.level: int = level
        self.text: str = text
        self.table: pd.DataFrame = table if table is not None else pd.DataFrame()
        self.operation: str = operation
        self.parent: Optional['TreeNode'] = None
        self.operation_history: List[str] = []
        
        # 新增屬性用於改進功能
        self.node_id: str = self._generate_node_id()
        self.created_at: datetime = datetime.now()
        self.validation_errors: List[str] = []
        self.table_hash: str = self._calculate_table_hash()
        self.semantic_score: float = 0.0
        
    def _generate_node_id(self) -> str:
        """生成唯一節點ID"""
        content = f"{self.level}_{self.operation}_{datetime.now().isoformat()}"
        return hashlib.md5(content.encode()).hexdigest()[:8]
        
    def _calculate_table_hash(self) -> str:
        """計算表格內容的哈希值，用於檢測重複"""
        if self.table.empty:
            return ""
        try:
            return hashlib.md5(str(self.table.values.tobytes()).encode()).hexdigest()[:8]
        except:
            return ""
    
    def add_child(self, child: 'TreeNode'):
        """添加子節點並進行驗證"""
        if self._validate_child(child):
            child.parent = self
            self.children.append(child)
            logger.info(f"添加子節點: {child.node_id} to {self.node_id}")
        else:
            logger.warning(f"子節點驗證失敗: {child.validation_errors}")
    
    def _validate_child(self, child: 'TreeNode') -> bool:
        """驗證子節點的合理性"""
        errors = []
        
        # 檢查是否有重複的表格狀態
        if child.table_hash and child.table_hash == self.table_hash:
            if not child.operation.lower().startswith('write'):
                errors.append("表格內容未發生變化但非寫作操作")
        
        # 檢查操作是否邏輯合理
        if self._is_redundant_operation(child.operation):
            errors.append(f"檢測到冗餘操作: {child.operation}")
        
        child.validation_errors = errors
        return len(errors) == 0
    
    def _is_redundant_operation(self, operation: str) -> bool:
        """檢查操作是否冗餘"""
        if len(self.operation_history) < 2:
            return False
            
        # 檢查是否有相同操作在近期歷史中
        recent_ops = self.operation_history[-3:]  # 檢查最近3個操作
        op_name = operation.split('(')[0].lower()
        
        for hist_op in recent_ops:
            if hist_op.split('(')[0].lower() == op_name:
                return True
        return False
    
    def is_leaf(self) -> bool:
        """判斷是否為葉節點"""
        return len(self.children) == 0
    
    def to_dict(self) -> Dict[str, Any]:
        """將節點轉換為字典格式，用於可視化"""
        return {
            "node_id": self.node_id,
            "level": self.level,
            "operation": self.operation,
            "text_preview": self.text[:100] + "..." if len(self.text) > 100 else self.text,
            "table_shape": list(self.table.shape) if not self.table.empty else [0, 0],
            "table_columns": list(self.table.columns) if not self.table.empty else [],
            "children_count": len(self.children),
            "validation_errors": self.validation_errors,
            "semantic_score": self.semantic_score,
            "created_at": self.created_at.isoformat(),
            "table_hash": self.table_hash
        }

# ===== 改進的操作解析器 =====
class OperationParser:
    """專門負責解析和驗證操作的類別"""
    
    def __init__(self):
        self.valid_operations = {
            'select_column', 'select_row',  'sort', 'calculate',
            'group_by', 'value_counts', 'aggregate', 'crosstab','pivot_table', 'write'
        }
        
    def parse_operations(self, response_text: str) -> List[Dict[str, Any]]:
        """改進的操作解析，返回結構化結果"""
        try:
            parsed_operations = []
            
            # 多種解析策略
            operations = self._extract_operations_multiple_strategies(response_text)
            
            for op_str in operations:
                parsed_op = self._parse_single_operation(op_str)
                if parsed_op and self._validate_operation(parsed_op):
                    parsed_operations.append(parsed_op)
                else:
                    logger.warning(f"無效操作被忽略: {op_str}")
            
            return parsed_operations[:5]  # 限制最多5個操作
            
        except Exception as e:
            logger.error(f"解析操作失敗: {e}")
            return []
    
    def _extract_operations_multiple_strategies(self, text: str) -> List[str]:
        """使用多種策略提取操作"""
        operations = []
        
        # 策略1: 尋找方括號內容
        bracket_match = re.search(r'\[(.*?)\]', text, re.DOTALL)
        if bracket_match:
            content = bracket_match.group(1)
            # 使用正則提取函數調用格式
            pattern = r'([a-zA-Z_]+\([^)]*\))'
            ops = re.findall(pattern, content)
            operations.extend(ops)
        
        # 策略2: 逐行解析
        if not operations:
            lines = text.split('\n')
            for line in lines:
                line = line.strip()
                if line and not line.startswith('#') and '(' in line and ')' in line:
                    operations.append(line)
        
        # 策略3: 逗號分割
        if not operations:
            parts = text.replace('[', '').replace(']', '').split(',')
            for part in parts:
                part = part.strip()
                if part and '(' in part:
                    operations.append(part)
        
        return operations
    
    def _parse_single_operation(self, op_str: str) -> Optional[Dict[str, Any]]:
        """解析單個操作字符串"""
        try:
            # 移除多餘的字符
            op_str = op_str.strip().rstrip(',').strip()
            
            # 提取操作名稱和參數
            if '(' not in op_str:
                return {"name": op_str, "args": [], "raw": op_str}
            
            name_part = op_str.split('(')[0].strip()
            args_part = op_str[op_str.find('(')+1:op_str.rfind(')')].strip()
            
            # 解析參數
            args = []
            if args_part:
                # 簡單的參數分割（可以進一步改進）
                for arg in args_part.split(','):
                    arg = arg.strip().strip('\'"')
                    if arg:
                        args.append(arg)
            
            return {
                "name": name_part.lower(),
                "args": args,
                "raw": op_str
            }
            
        except Exception as e:
            logger.error(f"解析操作 '{op_str}' 失敗: {e}")
            return None
    
    def _validate_operation(self, operation: Dict[str, Any]) -> bool:
        """驗證操作的有效性"""
        name = operation.get("name", "").lower()
        
        # 檢查操作名稱是否有效
        if name not in self.valid_operations:
            logger.warning(f"未知操作: {name}")
            return False
        
        # 檢查特定操作的參數
        args = operation.get("args", [])
        
        if name in ['select_column', 'sort_values'] and not args:
            logger.warning(f"{name} 操作需要參數")
            return False
        
        return True

# ===== 改進的內容規劃器 =====
class ContentPlanner:
    def __init__(self, api_key):
        self.api_key = api_key
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel("gemini-2.0-flash")
        self.parser = OperationParser()
        
    def generate_operations(self, tables, table_description, operation_description, 
                          operation_history, operation_pool, max_depth=5, max_degree=3, outline_path='main.txt'):
        """
        改進的操作生成，加入重複檢測和語意驗證
        """
        
        # 檢測近期操作，避免重複
        recent_operations = self._extract_recent_operations(operation_history)
        
        # 構建改進的提示詞
        prompt = f"""System : You are a content planner for the report. Please follow the outline. Please select candidate Operations and corresponding Arguments from the Operation Pool based on the input Tables and Operation History. These candidate Operations will be the next Operation in the Operation History .

# Requirements
1. Strictly adhere to the requirements .
2. The output must be in English .
3. The output must be based on the input data ; do not hallucinate .
4. The length of Operation History must be less than or equal to {max_depth}.
5. The number of Operations must be less than or equal to {max_degree}  and more than zero.
6. Only select Opertions from the Operation Pool .
7. Arguments must match the format required by the corresponding Operations .
8. Operations & Arguments must follow this format : [ operation_1 ( argument_1 , ...) , operation_2 ( argument_2 , ...) , operation_3 ( argument_3 , ...) , ...]
9. Only output Operations & Arguments !
10. If Table is big or Level is low, it should be more Operations include select_col or select_row not write.
11. If the length of Operation History is short, then more operations or more arguments.
12. Write operations do not need argument.
13. AVOID repeating recent operations: {recent_operations}
14. Prioritize operations that will meaningfully transform the data.
15. Avoid give the arguments that not match by the operation.

#outline
{read_text_file(outline_path) if os.path.exists(outline_path) else "Generate comprehensive data analysis"}

# Table Description
{table_description}

# Operation Description
{json.dumps(operation_description, indent=2, ensure_ascii=False)}

User : # Test
## Tables
{tables}

## Operation History
{operation_history}

## Operation Pool
{operation_pool}

## Operations & Arguments"""

        try:
            logger.info("正在向Gemini發送請求...")
            response = self.model.generate_content(prompt)
            
            if response.text:
                logger.info("成功獲得Gemini回應")
                parsed_ops = self.parser.parse_operations(response.text.strip())
                return [op["raw"] for op in parsed_ops]  # 返回原始字符串格式
            else:
                logger.warning("Gemini回應為空")
                return []
                
        except Exception as e:
            logger.error(f"Gemini API請求失敗: {e}")
            return []
    
    def _extract_recent_operations(self, operation_history: List[str]) -> List[str]:
        """提取最近的操作名稱"""
        recent = []
        for op in operation_history[-3:]:  # 最近3個操作
            if '(' in op:
                name = op.split('(')[0].strip()
                recent.append(name)
        return recent

# ===== 安全的DataFrame操作器 =====
class SafeDataFrameOperator:
    """安全的DataFrame操作器，使用AST驗證而非直接exec"""
    
    def __init__(self, api_key):
        self.api_key = api_key
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel("gemini-2.0-flash")
        self.allowed_modules = {'pandas', 'numpy', 're'}
        self.allowed_functions = {
            'pd.read_csv', 'pd.DataFrame', 'df.head', 'df.tail', 'df.sort_values',
            'df.groupby', 'df.filter', 'df.select', 'df.drop', 'df.fillna',
            'df.to_csv', 'df.value_counts', 'df.describe', 'df.info'
        }

    def generate_code(self, operation, df_info, df_path="input_tmp.csv"):
        prompt = f"""
        你是一個專業的Python資料分析助手。欄位名稱以資料欄位類型提供為主，根據以下要求生成操作DataFrame的程式碼：

        要執行的操作: {operation}

        CSV數據集: {df_path}

        資料欄位類型:
        {df_info}

        生成要求：
        1. 讀取CSV數據集，並存入DataFrame後，使用要執行的操作後，將修改後的DataFrame存入'tmp.csv'
        2. 只使用pandas基本操作，避免複雜的自定義函數
        3. 確保代碼安全，不包含文件系統操作（除了指定的CSV讀寫）
        4. 撰寫完整python code，包含錯誤處理

        輸出格式：
        ```python
        # 你的程式碼
        ```
        """
        return self._retry_generate(prompt)

    def _retry_generate(self, prompt, max_retries=2):
        """帶重試的生成請求"""
        for attempt in range(max_retries):
            try:
                response = self.model.generate_content(prompt)
                if response.text:
                    return response.text.strip()
            except Exception as e:
                logger.warning(f"生成代碼失敗 (嘗試 {attempt+1}/{max_retries}): {e}")
                if attempt < max_retries - 1:
                    import time
                    time.sleep(1)
        return ""

    def safe_execute(self, code: str, df: pd.DataFrame) -> pd.DataFrame:
        """安全執行生成的代碼"""
        try:
            # 提取代碼塊
            code_block = re.search(r'```python\n(.*?)\n```', code, re.DOTALL)
            #print(f'python code: {code_block}')
            if code_block:
                code = code_block.group(1)

            # AST安全驗證
            if not self._validate_code_safety(code):
                logger.error("代碼安全驗證失敗")
                return df

            # 寫入暫存 CSV 檔案
            df.to_csv("input_tmp.csv", index=False)

            allowed_builtin_names = [
                'int', 'float', 'str', 'bool', 'list', 'dict', 'set', 'tuple',
                'len', 'range', 'enumerate', 'zip', 'min', 'max', 'sum', 'abs',
                'print',
                'Exception', 'TypeError', 'ValueError', 'KeyError', 'IndexError',
                'FileNotFoundError', 'ZeroDivisionError', 'AttributeError', 'ImportError',
                '__import__'
            ]

            safe_globals = {
                'pd': pd,
                '__name__': '__main__',
                '__builtins__': {name: getattr(builtins, name) for name in allowed_builtin_names}
            }

            safe_locals = {}

            # 執行代碼
            exec(code, safe_globals, safe_locals)

            # 讀取結果
            if os.path.exists("tmp.csv"):
                result_df = pd.read_csv("tmp.csv")
                logger.info(f"操作成功，結果形狀: {result_df.shape}")
                return result_df
            else:
                logger.warning("未生成結果文件，返回原始DataFrame")
                return df

        except Exception as e:
            error_msg = f"執行錯誤: {str(e)}"
            print(error_msg)
            print("錯誤代碼如下：\n" + "-" * 30)
            print(code)  # ✅ 輸出造成錯誤的程式碼
            print("-" * 30)
            logger.error(error_msg)
            sys.exit(1)



    def _validate_code_safety(self, code: str) -> bool:
        """使用AST驗證代碼安全性"""
        try:
            tree = ast.parse(code)
            
            for node in ast.walk(tree):
                # 檢查危險的函數調用
                if isinstance(node, ast.Call):
                    if isinstance(node.func, ast.Name):
                        func_name = node.func.id
                        if func_name in ['exec', 'eval', 'compile', '__import__', 'open']:
                            logger.error(f"檢測到危險函數: {func_name}")
                            return False
                
                # 檢查文件操作（除了允許的CSV操作）
                if isinstance(node, ast.Call) and isinstance(node.func, ast.Attribute):
                    if hasattr(node.func, 'attr'):
                        attr_name = node.func.attr
                        if attr_name in ['system', 'popen', 'subprocess']:
                            logger.error(f"檢測到系統調用: {attr_name}")
                            return False
                
                # 檢查導入語句
                if isinstance(node, ast.Import):
                    for alias in node.names:
                        if alias.name not in self.allowed_modules:
                            logger.error(f"檢測到不允許的模組導入: {alias.name}")
                            return False
            
            return True
            
        except SyntaxError as e:
            logger.error(f"代碼語法錯誤: {e}")
            return False
        except Exception as e:
            logger.error(f"AST驗證失敗: {e}")
            return False

# ===== 文本生成器 =====
import time

class TextGenerator:
    def __init__(self, api_key, table_description=""):
        self.api_key = api_key
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel("gemini-2.0-flash")
        self.table_description = table_description

    def extract_highlights_from_table(self, table: pd.DataFrame) -> str:
        try:
            if 'lose_reason' in table.columns:
                top_reason = table['lose_reason'].value_counts().idxmax()
            else:
                top_reason = "無資料"
            if 'getpoint_player' in table.columns:
                top_player = table['getpoint_player'].value_counts().idxmax()
            else:
                top_player = "未知球員"
            return f"最多失分原因為「{top_reason}」，得分最多的是 {top_player}。"
        except:
            return ""

    def extract_table_features(self, table: pd.DataFrame) -> str:
        summary = []
        for col in table.columns:
            dtype = str(table[col].dtype)
            line = f"欄位「{col}」類型：{dtype}"

            # 顯示常見值僅限類別型欄位
            if table[col].nunique() <= 10 or dtype == 'object' or pd.api.types.is_categorical_dtype(table[col]):
                top_values = table[col].value_counts().head(3).to_dict()
                line += f"，常見值：{list(top_values.keys())}"
            summary.append(line)
        return "\n".join(summary)

    def _retry_generate(self, prompt, max_retries=3, delay_seconds=30):
        for attempt in range(max_retries):
            try:
                response = self.model.generate_content(prompt)
                if response.text:
                    return response.text.strip()
            except Exception as e:
                err = str(e)
                logger.error(f"Gemini 回應失敗: {err}")
                if "429" in err:
                    logger.info(f"已達配額限制，等待 {delay_seconds} 秒後重試 ({attempt+1}/{max_retries})...")
                    time.sleep(delay_seconds)
                else:
                    break
        return "⚠️ 寫作請求失敗：API 限制或其他錯誤"

    def generate_text_for_write_operation(self, table: pd.DataFrame, operation_history: List[str]) -> str:
        table_str = table.to_string()
        WRITE_TOKENS = 50
        TABLE_FORMAT = "Pandas DataFrame as plain text"
        highlight_summary = self.extract_highlights_from_table(table)
        table_feature_summary = self.extract_table_features(table)

        prompt = f"""
System :
You are a professional content writer for the badminton game report .
Please write the Report based on the input Table, just pick one or two lightspots.

# Requirements
1. Strictly adhere to the requirements .
2. The output must be in 中文 .
3. The output must be based on the input data ; do not hallucinate .
4. The Table format is {TABLE_FORMAT}.
5. The Report can only describe the content included in the Tables and cannot describe anything not included in the Tables .
6. The Report must consist of only one paragraph .
7. The number of tokens in the Report must be within {WRITE_TOKENS}.
8. 請專注描述得分與失分模式、關鍵欄位趨勢或球員亮點。
9. 請模仿比賽轉播員或教練的語氣描述，句式自然、有節奏感。
10. 請特別觀察球種之間的連續轉換，例如 放小球 接 殺球 等，找出其中有效得分或不尋常的組合並描述。

# Highlights Summary
{highlight_summary}

# Table Features
{table_feature_summary}

# Table Description
{self.table_description}

User :
# Test
## Tables
{table_str}
## Report
"""
        return self._retry_generate(prompt)

    def merge_child_texts(self, child_texts: List[str], parent_operation: str) -> str:
        if not child_texts:
            return ""

        GENERATING_TOKENS = 100
        reports_str = "\n".join([f"- {txt}" for txt in child_texts])
        prompt = f"""
System :
You are a content generator for the badminton game report .
Please merge and rewrite a New Report based on the input Reports .

# Requirements
1. Strictly adhere to the requirements .
2. The output must be in 中文 .
3. The output must be based on the input data ; do not hallucinate .
4. The New Report must include all the content from the input Reports ; do not omit any information .
5. The New Report must follow the order of the input Reports .
6. The number of tokens in the New Report must be within {GENERATING_TOKENS}.
7. 請依序整合每段內容，形成結構清晰的段落，包括亮點、失誤模式與球員貢獻。

User :
# Test
## Reports
{reports_str}
## New Report
"""
        return self._retry_generate(prompt)

# ===== OperationParser._validate_operation 強化參數驗證（補入 df 欄位比對） =====
def validate_operation_with_columns(operation: Dict[str, Any], df_columns: List[str]) -> bool:
    name = operation.get("name", "").lower()
    args = operation.get("args", [])

    # 檢查操作名稱是否有效
    if name not in {
        'select_column', 'select_row', 'sort', 'calculate',
        'group_by', 'value_counts', 'aggregate', 'crosstab', 'pivot_table', 'write'
    }:
        return False

    # 僅針對需參數操作檢查欄位
    if name in ['select_column', 'sort', 'group_by']:
        for arg in args:
            if arg not in df_columns:
                return False

    return True



# ===== 改進的TreeOfReport類別 =====
class TreeOfReport:
    def __init__(self, api_key: str, max_depth: int = 5, max_degree: int = 5):
        self.api_key = api_key
        self.max_depth = max_depth
        self.max_degree = max_degree

        # 載入配置檔案
        self.load_configurations()

        # 初始化改進的組件
        self.content_planner = ContentPlanner(api_key)
        self.df_operator = SafeDataFrameOperator(api_key)  # 使用安全版本
        self.text_generator = TextGenerator(api_key, table_description=self.table_description)
        
        # 新增追蹤功能
        self.execution_log: List[Dict[str, Any]] = []
        self.node_registry: Dict[str, TreeNode] = {}

    def load_configurations(self):
        self.table_description = read_text_file("filtered_data _description.txt")
        if not self.table_description or self.table_description == "No file available":
            self.table_description = "數據分析表格，包含各種欄位用於分析"

        self.operation_description = read_json_file("selected_operations.json")
        if isinstance(self.operation_description, list):
            self.operation_pool = [op['name'] for op in self.operation_description]
        else:
            self.operation_pool = list(self.operation_description.keys())

        logger.info(f"載入操作池: {self.operation_pool}")

    def build_tree(self, root_table: pd.DataFrame) -> TreeNode:
        """改進的樹構建，加入完整的追蹤和驗證"""
        root = TreeNode(level=0, text="資料分析報告", table=root_table, operation="root(None)")
        root.operation_history = ['root(None)']
        self.node_registry[root.node_id] = root
        
        queue = [root]
        
        while queue:
            current_node = queue.pop(0)
            
            # 記錄處理日誌
            self._log_node_processing(current_node)

            if current_node.operation.lower().startswith('write'):
                continue

            if current_node.level >= self.max_depth:
                write_node = self.create_child_node(current_node, 'write()')
                if write_node:
                    current_node.add_child(write_node)
                continue

            logger.info(f"處理節點 - Level: {current_node.level}, Operation: {current_node.operation}")

            tables_str = current_node.table.to_string()
            operations = self.content_planner.generate_operations(
                tables=tables_str,
                table_description=self.table_description,
                operation_description=self.operation_description,
                operation_history=current_node.operation_history,
                operation_pool=self.operation_pool,
                max_depth=self.max_depth,
                max_degree=self.max_degree
            )

            logger.info(f"生成操作: {operations}")

            for operation in operations[:self.max_degree]:
                if operation.strip():
                    child_node = self.create_child_node(current_node, operation)
                    if child_node:
                        current_node.add_child(child_node)
                        queue.append(child_node)

        self.generate_all_texts(root)
        return root
    
    def _log_node_processing(self, node: TreeNode):
        """記錄節點處理日誌"""
        log_entry = {
            "timestamp": datetime.now().isoformat(),
            "node_id": node.node_id,
            "level": node.level,
            "operation": node.operation,
            "table_shape": list(node.table.shape) if not node.table.empty else [0, 0],
            "validation_errors": node.validation_errors
        }
        self.execution_log.append(log_entry)
    
    def create_child_node(self, parent: TreeNode, operation: str) -> Optional[TreeNode]:
        """改進的子節點創建，加入完整驗證"""
        try:
            # 建立新的操作歷史
            new_operation_history = parent.operation_history + [operation]
            
            # 檢查是否為 write 操作
            if operation.lower().startswith('write'):
                text = self.text_generator.generate_text_for_write_operation(
                    parent.table,
                    new_operation_history
                )
                child = TreeNode(
                    level=parent.level + 1,
                    text=text,
                    table=parent.table.copy(),
                    operation=operation
                )
                child.operation_history = new_operation_history
                self.node_registry[child.node_id] = child
                logger.info(f"創建 write 節點: {operation}")
                return child
            else:
                # 其他操作：執行數據操作
                df_info = f"Shape: {parent.table.shape}\nColumns: {list(parent.table.columns)}\nData types:\n{parent.table.dtypes.to_string()}"
                code = self.df_operator.generate_code(operation, df_info)
                
                if code:
                    result_df = self.df_operator.safe_execute(code, parent.table)
                    child = TreeNode(
                        level=parent.level + 1,
                        text="",
                        table=result_df,
                        operation=operation
                    )
                    child.operation_history = new_operation_history
                    self.node_registry[child.node_id] = child
                    logger.info(f"創建數據操作節點: {operation}, 結果形狀: {result_df.shape}")
                    return child
                else:
                    logger.warning(f"無法生成操作代碼: {operation}")
                    return None
        
        except Exception as e:
            logger.error(f"創建子節點失敗: {e}")
            return None
    
    def generate_all_texts(self, node: TreeNode):
        """遞歸生成所有節點的文本"""
        for child in node.children:
            self.generate_all_texts(child)
        
        if node.is_leaf() and not node.text and node.operation and not node.operation.lower().startswith('write'):
            node.text = self.text_generator.generate_text_for_write_operation(
                node.table, 
                node.operation_history
            )
            print(f'node table: {node.table}')
        elif node.children:
            child_texts = [child.text for child in node.children if child.text.strip()]
            if child_texts:
                merged_text = self.text_generator.merge_child_texts(
                    child_texts, 
                    node.operation or "root"
                )
                if node.text:
                    node.text = node.text + "\n\n" + merged_text
                else:
                    node.text = merged_text
        logger.info(f'節點 {node.node_id} 文本生成完成')
        print(f'node.table: {node.table}')
        print(f'節點文本: {node.text}')
        
    def export_tree_structure(self, root: TreeNode, output_path: str = "tree_structure.json"):
        """導出樹結構為JSON格式，用於可視化和分析"""
        def node_to_dict(node: TreeNode) -> Dict[str, Any]:
            result = node.to_dict()
            result["children"] = [node_to_dict(child) for child in node.children]
            return result
        
        tree_data = {
            "metadata": {
                "export_time": datetime.now().isoformat(),
                "total_nodes": len(self.node_registry),
                "max_depth": self.max_depth,
                "max_degree": self.max_degree
            },
            "execution_log": self.execution_log,
            "tree": node_to_dict(root)
        }
        
        try:
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(tree_data, f, indent=2, ensure_ascii=False)
            logger.info(f"樹結構已導出至: {output_path}")
        except Exception as e:
            logger.error(f"導出樹結構失敗: {e}")
    
    def generate_execution_report(self) -> str:
        """生成執行過程報告"""
        total_nodes = len(self.node_registry)
        error_nodes = sum(1 for node in self.node_registry.values() if node.validation_errors)
        
        report = f"""
# Tree-of-Report 執行報告

## 統計信息
- 總節點數: {total_nodes}
- 錯誤節點數: {error_nodes}
- 樹最大深度: {self.max_depth}
- 最大分支度: {self.max_degree}

## 節點分布
"""
        
        # 按層級統計節點
        level_counts = {}
        for node in self.node_registry.values():
            level = node.level
            level_counts[level] = level_counts.get(level, 0) + 1
        
        for level, count in sorted(level_counts.items()):
            report += f"- Level {level}: {count} 個節點\n"
        
        # 錯誤摘要
        if error_nodes > 0:
            report += "\n## 驗證錯誤摘要\n"
            for node in self.node_registry.values():
                if node.validation_errors:
                    report += f"- 節點 {node.node_id} ({node.operation}): {'; '.join(node.validation_errors)}\n"
        
        return report

    def generate_report(self, node: TreeNode, level: int = 0) -> str:
        """改進的報告生成"""
        if node.level == 0:
            prompt = f"""
            你是一位新聞記者，根據以下分析總結，請撰寫一篇賽事新聞報導，提供全面深入的分析，統整成新聞報導，文辭中過多直接使用欄位名稱與直接次數統計，用player_A與player_B表示兩球員，用生動的文句描述，勿出現累贅的句子，請從分析總結中提取轉換，禁止出現幻覺。
            請用繁體中文撰寫，保持邏輯清晰，資訊準確。

            分析總結:
            {node.text}
            """
            final_text = self.text_generator._retry_generate(prompt)
            
            # 保存多種格式的報告
            with open("tree_of_report.txt", "w", encoding="utf-8") as f:
                f.write(final_text)
            
            # 導出樹結構
            self.export_tree_structure(node)
            
            # 生成執行報告
            exec_report = self.generate_execution_report()
            with open("execution_report.md", "w", encoding="utf-8") as f:
                f.write(exec_report)
                print("finish generate report")
            
            return final_text
        else:
            logger.info(f'generate report from not root')
            indent = "  " * level
            report = f"{indent}{'#' * (level + 1)} {node.operation or 'Root'}\n\n"

            if node.text:
                report += f"{indent}{node.text}\n\n"

            if node.table is not None and not node.table.empty and level < 2:
                report += f"{indent}**資料摘要:** Shape {node.table.shape}\n"
                if len(node.table) <= 10:
                    report += f"{indent}```\n{node.table.to_string()}\n{indent}```\n\n"
                else:
                    report += f"{indent}```\n{node.table.head().to_string()}\n{indent}```\n\n"

            for child in node.children:
                report += self.generate_report(child, level + 1)

            return report


# ===== 主程序 =====
def main():
    """改進的主函數"""
    
    # 設置API密鑰
    api_key = os.getenv("Gemini_API")
    if not api_key:
        logger.error("請設置 Gemini_AP 環境變數")
        return
    
    logger.info("Tree-of-Report for Data Analysis (改進版)")
    logger.info("="*50)
    
    logger.info("正在載入數據...")
    
    # 讀取CSV檔案

    TABLES = pd.read_csv('filtered_set1.csv')
    logger.info(f"成功載入CSV: {TABLES.shape[0]} 行, {TABLES.shape[1]} 列")

    
    # 設置參數
    MAX_DEPTH = 3
    MAX_DEGREE = 4
    
    logger.info(f"最大深度: {MAX_DEPTH}")
    logger.info(f"最大分支度: {MAX_DEGREE}")
    
    # 初始化改進的 Tree-of-Report
    tree_report = TreeOfReport(api_key, max_depth=MAX_DEPTH, max_degree=MAX_DEGREE)
    
    # 建構報告樹
    logger.info("開始建構報告樹...")
    start_time = datetime.now()
    
    try:
        root = tree_report.build_tree(TABLES)
        
        # 生成最終報告
        logger.info("生成最終報告...")
        final_report = tree_report.generate_report(root)
        
        # 輸出報告
        logger.info("\n" + "="*50)
        logger.info("TREE-OF-REPORT 最終報告")
        logger.info("="*50)
        print(final_report)
        
        # 儲存報告
        with open('tree_of_report.md', 'w', encoding='utf-8') as f:
            f.write("# Tree-of-Report 數據分析報告 (改進版)\n\n")
            f.write(final_report)
        
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds()
        
        logger.info(f"報告生成完成，耗時: {duration:.2f} 秒")
        logger.info("生成的文件:")
        logger.info("- tree_of_report.md: 最終報告")
        logger.info("- tree_of_report.txt: 純文本報告")
        logger.info("- tree_structure.json: 樹結構數據")
        logger.info("- execution_report.md: 執行過程報告")
        logger.info("- tree_visualization.html: 可視化頁面")
        
    except Exception as e:
        logger.error(f"程序執行失敗: {e}")
        import traceback
        traceback.print_exc()
    
    finally:
        # 清理暫存檔案
        for temp_file in ['input_tmp.csv', 'tmp.csv']:
            if os.path.exists(temp_file):
                try:
                    os.remove(temp_file)
                    logger.info(f"清理暫存檔案: {temp_file}")
                except:
                    pass

if __name__ == "__main__":
    main()

2025-09-28 18:24:28,229 - INFO - Tree-of-Report for Data Analysis (改進版)
2025-09-28 18:24:28,230 - INFO - ==================================================
2025-09-28 18:24:28,231 - INFO - 正在載入數據...
2025-09-28 18:24:28,234 - INFO - 成功載入CSV: 315 行, 9 列
2025-09-28 18:24:28,235 - INFO - 最大深度: 3
2025-09-28 18:24:28,235 - INFO - 最大分支度: 4
2025-09-28 18:24:28,237 - INFO - 載入操作池: ['description', 'requirements', 'operations']
2025-09-28 18:24:28,238 - INFO - 開始建構報告樹...
2025-09-28 18:24:28,239 - INFO - 處理節點 - Level: 0, Operation: root(None)
2025-09-28 18:24:28,261 - INFO - 正在向Gemini發送請求...
2025-09-28 18:24:30,509 - INFO - 成功獲得Gemini回應
2025-09-28 18:24:30,510 - INFO - 生成操作: ['value_counts(type)', 'value_counts(lose_reason)', 'value_counts(getpoint_player)']
2025-09-28 18:24:32,694 - INFO - 操作成功，結果形狀: (18, 2)
2025-09-28 18:24:32,695 - INFO - 創建數據操作節點: value_counts(type), 結果形狀: (18, 2)
2025-09-28 18:24:32,695 - INFO - 添加子節點: 80f7ab6b to 74b9da25


value_counts('type') 操作完成，結果已儲存到 tmp.csv


2025-09-28 18:24:34,318 - INFO - 操作成功，結果形狀: (4, 2)
2025-09-28 18:24:34,318 - INFO - 創建數據操作節點: value_counts(lose_reason), 結果形狀: (4, 2)
2025-09-28 18:24:34,319 - INFO - 添加子節點: a03b196e to 74b9da25


value_counts 操作完成，結果已儲存至 tmp.csv


2025-09-28 18:24:36,517 - INFO - 操作成功，結果形狀: (2, 2)
2025-09-28 18:24:36,518 - INFO - 創建數據操作節點: value_counts(getpoint_player), 結果形狀: (2, 2)
2025-09-28 18:24:36,519 - INFO - 添加子節點: a7746a65 to 74b9da25
2025-09-28 18:24:36,519 - INFO - 處理節點 - Level: 1, Operation: value_counts(type)
2025-09-28 18:24:36,521 - INFO - 正在向Gemini發送請求...


...

node.table:   getpoint_player  count
0               A     21
1               B     15
節點文本: A隊與B隊比分緊咬，互不相讓，各得一分！A隊在本場比賽中以21分略勝B隊的15分。A隊今天在場上展現強勁進攻火力，遠超B隊。A隊員在多拍相持中總能抓住機會得分，奠定勝局。雙方得分模式有待加強，部分數據缺失，需進一步分析原因。


2025-09-28 18:27:11,932 - INFO - 節點 74b9da25 文本生成完成
2025-09-28 18:27:11,936 - INFO - 生成最終報告...


node.table:      Unnamed: 0  rally      time  roundscore_A  roundscore_B player  type  \
0             0      1  00:05:47             1             0      B   發長球   
1             1      1  00:05:49             1             0      A    切球   
2             2      1  00:05:50             1             0      B    挑球   
3             3      1  00:05:51             1             0      A    長球   
4             4      1  00:05:52             1             0      B    殺球   
..          ...    ...       ...           ...           ...    ...   ...   
310         310     36  00:24:44            21            15      B  未知球種   
311         311     36  00:24:58            21            15      A    切球   
312         312     36  00:25:00            21            15      B    挑球   
313         313     36  00:25:01            21            15      A    長球   
314         314     36  00:25:02            21            15      B    長球   

    lose_reason getpoint_player  
0           NaN             N

2025-09-28 18:27:16,086 - INFO - 樹結構已導出至: tree_structure.json
2025-09-28 18:27:16,089 - INFO - 
2025-09-28 18:27:16,090 - INFO - TREE-OF-REPORT 最終報告
2025-09-28 18:27:16,090 - INFO - ==================================================
2025-09-28 18:27:16,099 - INFO - 報告生成完成，耗時: 167.86 秒
2025-09-28 18:27:16,100 - INFO - 生成的文件:
2025-09-28 18:27:16,100 - INFO - - tree_of_report.md: 最終報告
2025-09-28 18:27:16,101 - INFO - - tree_of_report.txt: 純文本報告
2025-09-28 18:27:16,102 - INFO - - tree_structure.json: 樹結構數據
2025-09-28 18:27:16,103 - INFO - - execution_report.md: 執行過程報告
2025-09-28 18:27:16,103 - INFO - - tree_visualization.html: 可視化頁面
2025-09-28 18:27:16,106 - INFO - 清理暫存檔案: input_tmp.csv
2025-09-28 18:27:16,107 - INFO - 清理暫存檔案: tmp.csv


finish generate report
## 強攻制勝！A隊險勝B隊，關鍵在於長球與殺球的精準運用

一場激烈的羽球對決昨日落幕，A隊以21比15的比分險勝B隊。縱觀全場，雙方你來我往，互不相讓，但A隊憑藉更具侵略性的進攻，最終成功鎖定勝局。

本場比賽的焦點在於球員們對長球和殺球的頻繁運用。球員們多次利用長球調動對手，伺機以雷霆萬鈞的殺球直取分數，總計36次的殺球得分證明了這一戰術的有效性。A隊在多拍相持中展現出更強的得分能力，往往能在關鍵時刻抓住機會，給予對手致命一擊。

然而，比賽中也暴露出一些問題。雙方球員都出現了非受迫性失誤，其中出界次數偏高，顯示球員在控球方面仍有提升空間。此外，一些失分原因不明，以及未知球種的出現，都提醒教練團隊需要深入分析比賽錄影，找出潛在的技術漏洞。

B隊方面，雖然在防守端做出了努力，但仍難以抵擋A隊如潮水般的攻勢。對手頻繁利用落地得分，使得B隊在防守上疲於奔命。更令人擔憂的是，B隊在“a”原因上失分較多，這將是他們未來訓練中需要重點檢討的環節。

值得一提的是，有"未知球員"在本場比賽中表現亮眼，多次得分，為比賽增添了不少看點。

總體而言，A隊憑藉更強大的進攻火力，特別是長球和殺球的有效配合，贏得了這場比賽的勝利。然而，雙方在得分模式上都有待加強，並需針對各自的弱點進行改進，期待他們在未來的比賽中帶來更精彩的表現。


In [84]:
import os
import pandas as pd
import numpy as np
import google.generativeai as genai
import time
from datetime import datetime

# === 寫作風格詞彙 ===
BADMINTON_TERMS = {
    'net': '網前失誤', 'out': '出界', 'long': '過底線', 'smash': '殺球',
    'clear': '高遠球', 'drop': '切球', 'drive': '平抽球', 'serve': '發球', 'return': '回球'
}
ACTION_VERBS = ['展現', '發揮', '掌握', '運用', '施展', '控制', '主導', '壓制', '突破', '創造', '締造', '奠定', '確立', '鞏固', '扭轉', '逆轉']
TECHNICAL_TERMS = ['lose_reason', 'getpoint_player', 'type', 'column', 'row']

# === Gemini 模型初始化 ===
def init_model(api_key: str):
    genai.configure(api_key=api_key)
    return genai.GenerativeModel("gemini-2.0-flash")

# === 品質評估 ===
def assess_text_quality(text: str) -> float:
    score = 0.0
    if 30 <= len(text) <= 120:
        score += 0.2
    score += min(0.2, sum(1 for t in BADMINTON_TERMS.values() if t in text) * 0.1)
    score += min(0.2, sum(1 for v in ACTION_VERBS if v in text) * 0.05)
    if not any(t in text for t in TECHNICAL_TERMS):
        score += 0.2
    if '，' in text or '。' in text:
        score += 0.2
    return round(min(score, 1.0), 2)

# === 主流程：重複3次生成並評估 ===
def generate_best_of_three(df: pd.DataFrame, api_key: str):
    model = init_model(api_key)
    table_str = df.to_string(index=False)

    prompt_template = f"""
你是一位專業體育新聞記者，擅長撰寫羽球比賽報導。
請根據以下數據表格撰寫賽事描述，使用繁體中文，避免出現技術欄位名稱。

# 賽事數據表格：
{table_str}

請撰寫描述：
"""

    results = []
    for i in range(3):
        try:
            print(f"⏳ 第 {i+1}/3 次生成...")
            response = model.generate_content(prompt_template)
            time.sleep(1)
            text = response.text.strip() if response.text else "⚠️ 無內容"
        except Exception as e:
            text = f"⚠️ 生成錯誤: {e}"
        score = assess_text_quality(text)
        results.append({'index': i+1, 'text': text, 'score': score})

    # 選出最佳結果
    best = max(results, key=lambda x: x['score'])

    # 輸出到檔案
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    file_name = f"best_of_three_report_{timestamp}.txt"
    with open(file_name, "w", encoding="utf-8") as f:
        for r in results:
            f.write(f"[版本 {r['index']}] 品質分數: {r['score']}\n{r['text']}\n\n")
        f.write(f"🏆 最佳版本為第 {best['index']} 次，分數: {best['score']}\n")
        f.write(best['text'])

    print("\n✅ 所有版本已生成")
    for r in results:
        print(f"[{r['index']}] 分數: {r['score']} → {r['text']}")
    print(f"\n🏆 最佳版本是第 {best['index']} 次：{best['text']}")
    print(f"✔️ 已儲存至：{file_name}")
    return best

# === 測試入口 ===
if __name__ == "__main__":
    api_key = os.getenv("Gemini_API")
    if not api_key:
        raise RuntimeError("請設置 Gemini_API 環境變數")

    df = pd.read_csv("filtered_set1.csv")
    

    generate_best_of_three(df, api_key)


⏳ 第 1/3 次生成...
⏳ 第 2/3 次生成...
⏳ 第 3/3 次生成...

✅ 所有版本已生成
[1] 分數: 0.75 → 這場羽球賽事可謂高潮迭起，雙方你來我往，互不相讓。從比賽伊始，雙方便展開了激烈的攻防轉換，發球、過渡球、到進攻，每個回合都充滿了變數。可以看到球員A率先取得領先，一路將比分拉開，一度取得11:6的優勢。然而，球員B並沒有輕易放棄，展現了頑強的韌性，逐漸將比分追趕上來。

比賽中，雙方選手都力圖在前場尋找機會，短球的運用頻繁，小球與挑球的搭配也考驗著雙方的技術。一些回合的拉鋸非常長，球員們不斷地進行攻防轉換，後場的強力擊球與前場的精巧控制相互交織，呈現出精彩的對抗場面。失誤也偶爾出現，掛網、出界等情況讓比賽更具懸念。

比賽後半段，球員B逐漸找到狀態，憑藉積極的跑動和抓住機會的能力，將比分反超，最終以21:15的比分贏得了勝利。整場比賽節奏緊湊，雙方都展現了高超的羽球技藝和頑強的鬥志，是一場值得回味的精彩對決。
[2] 分數: 0.6 → 這場羽球賽事戰況膠著，雙方你來我往，互不相讓。比賽初段，雙方都以試探性的發球開局，隨後球路變化多端，有時是輕巧的網前小球，有時是力道十足的後場重擊，看得出雙方選手都在積極尋找對方的破綻。

比賽中，選手A一度取得領先，但選手B韌性十足，緊咬比分。在多拍來回中，雙方都展現了極佳的防守能力，多次將看似必殺的球路化解。網前的細膩手法和後場的強力進攻交織，讓觀眾看得目不暇給。

在關鍵時刻，選手A利用一次精準的判斷，讓對手措手不及，成功得分。然而，選手B也毫不示弱，隨即以一記漂亮的落地得分還以顏色。比分交替上升，比賽氣氛也越發緊張。

最終，選手A穩住陣腳，憑藉著穩定的發揮和關鍵時刻的果斷進攻，成功拿下分數。但選手B的表現也同樣精彩，雖敗猶榮。整場比賽高潮迭起，充分展現了羽球運動的魅力。觀眾們也為這場精彩的對決獻上了熱烈的掌聲。
[3] 分數: 0.75 → 這場羽球賽事可謂高潮迭起，雙方選手你來我往，攻防轉換節奏快速。開局雙方互有領先，比分交替上升，首局前半段A選手稍佔優勢，一度將比分拉開至2:1，但B選手隨即展開反擊，利用精準的落點控制和強勢的進攻，將比分追平。

比賽中，我們可以看到多回合的精采對決。例如第三分，雙方選手經過多次的短球、挑球、長球、抽球、切球等戰術運用，足足來回了17拍才由A選手